In [82]:
import numpy as np
import random
import pandas as pd
import math
from random import randint

In [83]:
def _readfile(file):

    # ----- read data -----
    df = pd.read_csv(file)
    df = df.fillna(method = 'ffill')

    # ----- Create Features -----
    X = df.drop(['id','class'], axis=1).copy(deep=False)
    X = (X-X.min())/(X.max()-X.min())

    # ----- Create Desired outputs -----
    Y = df[['class']].copy(deep=False)

    Input = X.to_numpy()
    Output = Y.to_numpy()

    return Input,Output

Input,Output = _readfile('data/wdbc.csv')

In [84]:
def cross_validations_split(shape,folds):
    fold_size = int(shape * folds/100)
    k = 0
    index = []
    for i in range(1,folds+1):
        if i < folds:
            index.append([k,i*fold_size])
        else:
            index.append([k,shape])
        k = i*fold_size
    return index

In [85]:
class GA(object):
    def __init__(self, hiddenSize, inputSize, outputSize):

        # initiate layers
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize

        layers = [self.inputSize] + self.hiddenSize + [self.outputSize]
        # initiate genes
        genes = []
        for i in range(len(layers)-1):
            g = np.random.uniform(-2,2,(layers[i], layers[i+1]))
            genes.append(g)
        self.genes = genes
        self.layers = layers

   

    def feedForward(self, X):
        Output_node = X
        for i, g in enumerate(self.genes):
        
            v = np.dot(Output_node, g)
            Output_node = self.sigmoid(v)

        return Output_node

    def sigmoid(self, s, deriv=False):
        if (deriv == True):
            return s * (1-s)
        return 1/(1 + np.exp(-s))

    def object_funct(self, X, Y):

        seed = randint(1, 100*100)

        np.random.seed(seed)
        np.random.shuffle(X)

        np.random.seed(seed)
        np.random.shuffle(Y)

        sum_err = 0
        for j, input in enumerate(X):
            target = Y[j]
            output = self.feedForward(input)

            sum_err += self._mse(target, output)

        self.fx = 1/math.log(1+sum_err/len(X))
        
        return self.fx

    def _mse(self, target, output):
        return np.average(abs(target - output)**2)


In [86]:
class selection(object):
    def __init__(self, chromosomes):

        # initiate fitness
        self.chromosomes = chromosomes
        self.Max = 1.2
        self.Min = 2-self.Max
    
    # initiate probability
    def prob(self):

        P = []
        N = len(chromosomes)
        Max = self.Max
        Min = self.Min

        for r,_ in enumerate(self.chromosomes):
            p = (Min + (Max - Min) * ((r-1)/(N-1)))/N
            P.append(p)

        return P

    def expect_values(self):

        ni = []
        P = self.prob()
        
        for p in P:
            ni.append(p*len(self.chromosomes))

        return ni

    # Stochastic universal sampling
    def Stochastic_sampling(self):

        ni = self.expect_values()
        answers = []
        index = []
        ptr = np.random.uniform(0,1,1)[0]
        sum_pi = 0

        for i in range(len(self.chromosomes)):
            sum_pi += ni[i]
            #print(sum_pi," vs ",ptr)
            while sum_pi > ptr :
                #print("select : ",i)
                index.append(i)
                ptr+=1
        for i in index:
            #print(self.chromosomes[i])
            answers.append(self.chromosomes[i][1])
        return answers

In [87]:
def Crossover_chromosomes(mating_pool):
    
    new_generation = []
    
    t = random.randint(int(len(mating_pool)*0.8), len(mating_pool))

    for i in range(t):
        couple = random.sample(mating_pool, 2) 
        c1 = []
        c2 = []

        # flatten genes
        for i in range(len(couple[0].genes)):
            c1.append(couple[0].genes[i].flatten())
            c2.append(couple[1].genes[i].flatten())
        c1 = [item for sublist in c1 for item in sublist]
        c2 = [item for sublist in c2 for item in sublist]

        # crossover c1&c2
        c3 = []
        for i in range(len(c1)): 
            j = random.randint(1, 2)
            if j == 1:
                c3.append(c1[i])
            else:
                c3.append(c2[i])
            
        # layer genes
        start = 0
        end = 0
        Hidden,Input,Output = couple[0].hiddenSize,couple[0].inputSize,couple[0].outputSize
        child = GA(Hidden,Input,Output)
        layers = child.layers
        for i in range(len(layers)-1):
            end += (layers[i]*layers[i+1])
            g = np.array(c3[start:end])
            g = g.reshape((layers[i], layers[i+1]))
            child.genes[i] = g.copy()
            start = end

        # Append childs
        new_generation.append(child)

    while len(new_generation) < len(mating_pool):

        child = random.choice(mating_pool)
        new_generation.append(child)

    return new_generation


In [88]:
def Mutation(chromosomes):

    for c in chromosomes:
        for i, g in enumerate(c.genes):
            print(g.shape[1])

In [89]:
chromosomes = []
no_generations = 100
no_chromosomes = 40

In [90]:
# initiate chromosomes P(0)
for i in range(no_chromosomes):
    c = GA([20,10],Input.shape[1],Output.shape[1])
    chromosomes.append(c)
    
# initiate best objective function
best_chromosome = [float('-inf'),chromosomes[0]]

x_train, y_train = Input,Output

In [91]:
for t in range(no_generations):

    print("----------------------------- P[",t,"] -----------------------------")

    # evaluation objective function
    ranks = []
    for c in chromosomes:

        fx = c.object_funct(x_train, y_train)
        ranks.append([fx,c])

        if fx > best_chromosome[0]:
            print("--- update best fitenss ",round(fx,6)," ---")
            best_chromosome[0] = fx
            best_chromosome[1] = c

    # Linear ranks selection
    ranks = sorted(ranks, key = lambda x: (x[0]))
    mating_pool = selection(ranks).Stochastic_sampling()
    

    # Crossover 
    c_chromosomes = Crossover_chromosomes(mating_pool)
    Mutation(c_chromosomes)
    # Mutation

    break


----------------------------- P[ 0 ] -----------------------------
--- update best fitenss  3.679031  ---
--- update best fitenss  3.687677  ---
--- update best fitenss  5.815158  ---
33
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20)
(20, 10)
(10, 1)
(30, 20